# Classifier

meer grid based dingen toevoegen, nu gaan we niet hoger komen


In [10]:
from SimpleCV import *
from IPython.display import HTML
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import sklearn 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import cv2
import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm


In [2]:
df = pd.read_csv("../dataset-numpy/dataset_features.csv")
df.head()


# split df in data (X) and labels (y)
X, y = df.iloc[:,:-1], df.iloc[:,-1]

# create train and test data and labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.20, random_state=42)

In [3]:
min_max_scaler = preprocessing.MinMaxScaler() # Scale the training features
training_set = min_max_scaler.fit_transform(X_train)
# Scale the test features (using the same settings as the trainingset)
test_set = min_max_scaler.transform(X_test)

In [4]:
# #svm classifier
# clfSVM = svm.SVC(gamma='scale')
# clfSVM.fit(X_train, y_train)
# y_pred = gnb.predict(X_test)
# print "SVM :",clfSVM.score(X_test, y_test)

In [5]:
#Naive Bayes Gaussian
gnb = GaussianNB()
gnb.fit(training_set, y_train)
#y_pred = gnb.predict(X_test)
print "Gaussian :",gnb.score(test_set, y_test)

#Naive Bayes Bernoulli
bnb = BernoulliNB()
bnb.fit(training_set, y_train)
#y_pred = gnb.predict(X_test)
print "Bernoulli:",bnb.score(test_set, y_test)


Gaussian : 0.838541666667
Bernoulli: 0.90625


In [6]:
#Random Forest
# Create a random forest classifier
clfRF = RandomForestClassifier(n_jobs=2, random_state=0)
# Train the classifier to take the training features and learn how they relate to the training y
clfRF.fit(training_set, y_train)
print "Random forest:",clfRF.score(test_set, y_test)

#Decision tree
clfDT = DecisionTreeClassifier(random_state=0)
clfDT.fit(training_set, y_train)
print "Decision tree:",clfDT.score(test_set, y_test)

Random forest: 0.916666666667
Decision tree: 0.8671875


In [7]:
# KNN
# niet accurate, hoeveelheid neighbors:
# 5 = 0.841666666667
# 6 = 0.835416666667
# 7 = 0.8375
# 1 = 0.827083333333
# 2 = 0.835416666667
# 3 = 0.845833333333
# 4 = 0.847916666667
# 8 = 0.83125
# 9 = 0.83125
# 10 = 0.8375

# min_max_scaler = preprocessing.MinMaxScaler() # Scale the training features
# training_set = min_max_scaler.fit_transform(X_train)
# # Scale the test features (using the same settings as the trainingset)
# test_set = min_max_scaler.transform(X_test)

# Create a classifier with k = 5
clfKNN = KNeighborsClassifier(n_neighbors=10)
# Train the classifier with the training data and training labels
clfKNN.fit(training_set, y_train)
 
# Score the classifier
# (calculates the mean accuracy of the given test data) 
score = clfKNN.score(test_set, y_test)
print "Score KNN: ", score

Score KNN:  0.919270833333


In [12]:
from sklearn.metrics import classification_report

# dit was test van online, niet gebruiken maar zelf gridsearch toevoegen

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()


# Tuning hyper-parameters for precision
()
Best parameters set found on development set:
()
{'kernel': 'linear', 'C': 10}
()
Grid scores on development set:
()
0.628 (+/-0.055) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.001}
0.464 (+/-0.083) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.0001}
0.869 (+/-0.012) for {'kernel': 'rbf', 'C': 10, 'gamma': 0.001}
0.747 (+/-0.032) for {'kernel': 'rbf', 'C': 10, 'gamma': 0.0001}
0.899 (+/-0.019) for {'kernel': 'rbf', 'C': 100, 'gamma': 0.001}
0.907 (+/-0.013) for {'kernel': 'rbf', 'C': 100, 'gamma': 0.0001}
0.892 (+/-0.022) for {'kernel': 'rbf', 'C': 1000, 'gamma': 0.001}
0.933 (+/-0.020) for {'kernel': 'rbf', 'C': 1000, 'gamma': 0.0001}
0.947 (+/-0.008) for {'kernel': 'linear', 'C': 1}
0.950 (+/-0.009) for {'kernel': 'linear', 'C': 10}
0.950 (+/-0.009) for {'kernel': 'linear', 'C': 100}
0.950 (+/-0.009) for {'kernel': 'linear', 'C': 1000}
()
Detailed classification report:
()
The model is trained on the full development set.
The scores are computed on